## Import

In [13]:
!pip install timm

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 549 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 42.7 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.5.0 which is incompatible.
apache-beam 2.24.0 requires typing-extensions<3.8.0,>=3.7.0, but you have typing-extensions 4.5.0 which is incompatible.
You should 

In [1]:
import os
from collections import defaultdict
from tqdm.notebook import tqdm
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms

from PIL import Image
import cv2

from sklearn.metrics import f1_score, accuracy_score
import random

In [2]:
label_mapper = defaultdict(list)

In [3]:
# def without_split(path, name, func=lambda x: x):
#     global label_mapper
#     for folder in tqdm(os.listdir(path)):
#         # path/folder
#         if func(folder):
#             # path/folder = temp
#             temp = os.path.join(path, folder)
#             # path/folder/file
#             for filename in os.listdir(temp):
#                 # path/folder/file -> label_mapper[folder_name]으로 저장
#                 label_mapper[f"{folder}_{name}"].append(os.path.join(temp, filename))

# def with_split(path, name, func=lambda x: x):
#     global label_mapper
#     for folder2 in tqdm(os.listdir(path)):
#         # path/folder2
#         if func(folder2):
#             # path/folder2/folder
#             for folder in os.listdir(os.path.join(path, folder2)):
#                 # path/folder2/folder = temp
#                 temp = os.path.join(path, folder2, folder)
#                 # path/folder2/folder/file
#                 for filename in os.listdir(temp):
#                     # path/folder2/folder/file -> label_mapper[folder_name]으로 저장
#                     label_mapper[f"{folder}_{name}"].append(os.path.join(temp, filename))

In [4]:
df = pd.read_csv('./train.csv')

In [5]:
df['crash_ego'] = df['label'].apply(lambda x: 0 if x==0 else 1 if x<=6 else 2)
df['timing'] = df['label'].apply(lambda x: 1 if x in [1,3,5,7,9,11] else 2 if x in [2,4,6,8,10,12] else 0)
df['weather'] = df['label'].apply(lambda x: 1 if x in [1,2,7,8] else 2 if x in [3,4,9,10] else 3 if x in [5,6,11,12] else 0)

In [6]:
label_mapper

defaultdict(list, {})

In [7]:
df['image_path'] = df.sample_id.apply(lambda x: 'train_cap/'+x+'.jpg')

In [8]:
for i in df.weather.unique():
    for a in df[df.weather==i].image_path:
        label_mapper[f'weather_{i}'].append(a)

In [9]:
# label_mapper[f"{folder}_{name}"].append(os.path.join(path/folder/filename))

In [10]:
data = []
n = 0
for i, key in enumerate(tqdm(label_mapper)):
    for sub in label_mapper[key]:
        for p in ["jpg", "jpeg", "png"]:
            if sub.endswith(p):
                data.append([sub, i])
                break
        else:
            n += 1

  0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
train_df, valid_df = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [12]:
import timm

model = timm.create_model('convnext_tiny', pretrained=True,
                          num_classes=len(label_mapper), in_chans=3).cuda()
for x in model.parameters(): x.requires_grad = True

## Fixed RandomSeed

In [13]:
def set_seed():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

In [19]:
class AccidentDataset(Dataset):
    def __init__(self, data_prep, transform=None):
        self.data_prep = data_prep
        self.transform = transforms.Compose([transforms.ToTensor(),
                                             transforms.Normalize((0.485, 0.456, 0.406),
                                                                  (0.229, 0.224, 0.225))])
        self.size = (224, 224)

    def __getitem__(self, idx):
        try:
            path, label = self.data_prep[idx][0], self.data_prep[idx][1]
            image = Image.open(path)
            image = image.convert(mode='RGB').resize(self.size)
            if self.transform:
                image = self.transform(image)
            return True, image, torch.tensor(label).long()
        except:
            return False, torch.randn(3, 224, 224), torch.tensor(-1).long()
    
    def __len__(self):
        return len(self.data_prep)

In [20]:
set_seed()

train_dataset = AccidentDataset(train_df)
valid_dataset = AccidentDataset(valid_df)

In [21]:
set_seed()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, 
                                           pin_memory=True, num_workers=2)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=32,
                                           pin_memory=True, num_workers=2)

In [22]:
def train(model, criterion, optimizer, train_dataloader, test_dataloader, num_epoch=10, path="/content/", score_function=lambda x: x):
    val_acc_log = []
    
    for epoch in tqdm(range(num_epoch)):
        model.train()
        pbar = tqdm(train_dataloader)
        for flag, imgs, labels in pbar:
            ################################
            imgs = torch.stack([imgs[i] for i in range(len(flag))
                                if flag[i]])
            labels = torch.stack([labels[i] for i in range(len(flag))
                                  if flag[i]])
            if len(labels):
                optimizer.zero_grad()
                imgs = imgs.cuda()
                labels = labels.cuda()
                y_pred = model(imgs)
                loss = criterion(y_pred, labels)
                pbar.set_description(str(np.round(loss.cpu().detach().numpy(), decimals=6)))
                loss.backward()
                optimizer.step()
        
        full_path = os.path.join(path, f"{model.__class__.__name__}_epoch_{epoch + 1}.pth")
        torch.save(model.state_dict(), full_path)
        print(f"Model saved to: {full_path}")

        val_pred = []
        val_true = []
        model.eval()
        with torch.no_grad():
            for flag, imgs, labels in tqdm(test_dataloader):
                imgs = torch.stack([imgs[i] for i in range(len(flag))
                                    if flag[i]])
                labels = torch.stack([labels[i] for i in range(len(flag))
                                      if flag[i]])
                if len(labels):
                    imgs = imgs.cuda()
                    labels = labels.cuda()

                    pred = model(imgs)
                    val_pred.extend(pred.argmax(1).cpu().detach().numpy())
                    val_true.extend(labels.cpu().detach().numpy())

        val_acc_log.append(score_function(np.array(val_true), np.array(val_pred)))
        print('Epoch', epoch + 1, f'Val {score_function.__name__}:', val_acc_log[-1])
    return val_acc_log

In [24]:
set_seed()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.003)
val_acc_log = train(model, criterion, optimizer, train_loader, valid_loader, num_epoch=10, score_function=accuracy_score, path="model/")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Model saved to: model/ConvNeXt_epoch_1.pth


  0%|          | 0/17 [00:00<?, ?it/s]

Epoch 1 Val accuracy_score: 0.6574074074074074


...

Epoch 9 Val accuracy_score: 0.6574074074074074


  0%|          | 0/68 [00:00<?, ?it/s]

Model saved to: model/ConvNeXt_epoch_10.pth


  0%|          | 0/17 [00:00<?, ?it/s]

Epoch 10 Val accuracy_score: 0.6574074074074074


## Data Load

In [46]:
class_names = sorted(df['weather'].unique())
n_classes = len(class_names)

In [52]:
class_dis = list(df['weather'].value_counts().values)

#### Image Classification(Recognition)_CustomDataset

In [56]:
root_path = 'train_cap/'

In [57]:
# Initialize DataGenerator
train_gen = IDG(rescale=1./255, horizontal_flip=True, rotation_range=20, validation_split=0.2)

# Load Data
train_ds = train_gen.flow_from_directory(root_path, target_size=(256,256), class_mode="binary", subset='training', shuffle=True, batch_size=32)
valid_ds = train_gen.flow_from_directory(root_path, target_size=(256,256), class_mode="binary", subset='validation', shuffle=True, batch_size=32)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [12]:
data = list()
for n, i in enumerate(sorted(os.listdir('train_cap'))):
    image = 'train_cap/' + i
    weather = df.loc[n, 'weather']
    timing = df.loc[n, 'timing']
    data.append((weather, timing, image))
    
data = pd.DataFrame(data, columns = ['weather','timing','image_path'])

unique_wea= data['weather'].unique()
unique_tim= data['timing'].unique()

In [13]:
def process_image(img):
    try:
        pic = cv2.imread(img)
        pic = cv2.cvtColor(pic, cv2.COLOR_BGR2RGB)
        pic = cv2.resize(pic, (256, 256), interpolation=cv2.INTER_AREA)
        pic = np.array(pic, dtype=np.float32)
        return pic
    except:
        return np.zeros(shape=(256, 256, 3), dtype=np.float32)

In [14]:
list_of_paths = data["image_path"].to_numpy()
images = [process_image(path) for path in list_of_paths]
images = np.array(images, dtype=np.float32)
images.shape

(2698, 256, 256, 3)

In [24]:
unique_wea = list(unique_wea) # Use for indexing the labels - ("hail", 0), ("rainbow", 1)
wea = data["weather"].to_numpy()

In [26]:
unique_tim = list(unique_tim) # Use for indexing the labels - ("hail", 0), ("rainbow", 1)
tim = data["timing"].to_numpy()

#### Split into train and test

In [28]:
images_train_wea, images_test_wea, labels_train_wea, labels_test_wea = train_test_split(images, wea, test_size=0.2)

images_train_tim, images_test_tim, labels_train_tim, labels_test_tim = train_test_split(images, tim, test_size=0.2)

In [29]:
images_train_wea = torch.from_numpy(images_train_wea).permute(0, 3, 1, 2)
images_test_wea = torch.from_numpy(images_test_wea).permute(0, 3, 1, 2)
labels_train_wea = torch.from_numpy(labels_train_wea)
labels_test_wea = torch.from_numpy(labels_test_wea)

images_train_tim = torch.from_numpy(images_train_tim).permute(0, 3, 1, 2)
images_test_tim = torch.from_numpy(images_test_tim).permute(0, 3, 1, 2)
labels_train_tim = torch.from_numpy(labels_train_tim)
labels_test_tim = torch.from_numpy(labels_test_tim)

In [30]:
del images
_ = gc.collect()

In [31]:
# Hyperparams Configuration.
args = {
    'epoch_num': 150,     
    'lr': 1e-3,           # Learning Rate
    'weight_decay': 1e-3, # L2 Penalty
    'batch_size': 32
}

In [32]:
if torch.cuda.is_available():
    args["device"] = torch.device("cuda")
else:
    args["device"] = torch.device("cpu")
    
print(args["device"])

cuda


### Dataloader

In [33]:
class ImagesLabelsDataset(Dataset):
    def __init__(self, images_array, labels_array, scaler_feat=None, scaler_label=None):

        self.labels = labels_array
        self.images = images_array
        
    def __getitem__(self, idx):

        sample = self.images[idx]
        label  = self.labels[idx]

        return sample, label

    def __len__(self):
        return len(self.labels)

In [34]:
# - Dataset first - #
train_set_wea = ImagesLabelsDataset(images_train_wea, labels_train_wea)
test_set_wea = ImagesLabelsDataset(images_test_wea, labels_test_wea)

# - Dataloader - #
train_loader_wea = DataLoader(train_set_wea, args['batch_size'], shuffle=True)
test_loader_wea = DataLoader(test_set_wea, args['batch_size'], shuffle=False)

In [35]:
# - Dataset first - #
train_set_tim = ImagesLabelsDataset(images_train_tim, labels_train_tim)
test_set_tim = ImagesLabelsDataset(images_test_tim, labels_test_tim)

# - Dataloader - #
train_loader_tim = DataLoader(train_set_tim, args['batch_size'], shuffle=True)
test_loader_tim = DataLoader(test_set_tim, args['batch_size'], shuffle=False)

In [36]:
class ConvolutionalModel(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.net = nn.Sequential(
            ## ConvBlock 1
            nn.Conv2d(3, 6, kernel_size=4, stride=1, padding=0),        
            # Input: (b, 3, 256, 256) || Output: (b, 6, 250, 250)
            nn.BatchNorm2d(6),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=5, stride=5, padding=0),           
            # Input: (b, 6, 250, 250) || Output: (b, 6, 50, 50)

            ## ConvBlock 2
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),       
            # Input: (b, 6, 50, 50) || Output: (b, 16, 46, 46)
            nn.BatchNorm2d(16),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2, padding=0),           
            # Input: (b, 16, 46, 46) || Output: (b, 16, 23, 23)

            ## ConvBlock 3
            nn.Conv2d(16, 32, kernel_size=8, stride=1, padding=0),     
            # Input: (b, 16, 23, 23) || Output: (b, 32, 16, 16)
            nn.BatchNorm2d(32),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=4, stride=4, padding=0),           
            # Input: (b, 32, 16, 16) || Output: (b, 32, 4, 4)

            ## ConvBlock 4
            nn.Conv2d(32, 120, kernel_size=4, stride=1, padding=0),     
            # Input: (b, 32, 4, 4) || Output: (b, 120, 1, 1)
            nn.BatchNorm2d(120),
            nn.Tanh(),
            nn.Flatten(),  # flat to a vector                
            # Input: (b, 120, 1, 1) || Output: (b, 120*1*1) = (b, 120)
            
            nn.Dropout(p=0.32), # Avoid Overfitting
            ## DenseBlock
            nn.Linear(120, 84),                                         
            # Input: (b, 120) || Output: (b, 84)
            nn.Tanh(),
            nn.Linear(84, output_size)                                
            # Input: (b, 84) || Output: (b, 10)
        )
        
    def forward(self, X):
        output = self.net(X)
        return output

In [37]:
net_wea = ConvolutionalModel(len(unique_wea)).to(args["device"])
net_tim = ConvolutionalModel(len(unique_tim)).to(args["device"])

### Training

In [38]:
criterion_wea = nn.CrossEntropyLoss().to(args['device'])
optimizer_wea = torch.optim.Adam(net_wea.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

criterion_tim = nn.CrossEntropyLoss().to(args['device'])
optimizer_tim = torch.optim.Adam(net_tim.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

In [39]:
def train(train_loader, net, epoch, criterion, optimizer):
    net.train()
    start = time.time()
    
    epoch_loss = list()
    pred_list, label_list = np.array([]), np.array([])
    
    for batch in train_loader:
        image, label = batch
        
        # GPU Casting
        image = image.to(args["device"])
        label = label.to(args["device"])
        
        # Forward
        pred_label = net(image)
        loss = criterion(pred_label, label)
        epoch_loss.append(loss.cpu().data)
        
        _, pred = torch.max(pred_label, axis=1)
        pred_list = np.append(pred_list, pred.cpu().numpy())
        label_list = np.append(label_list, label.cpu().numpy())
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    epoch_loss = np.asarray(epoch_loss)
    epoch_acc = accuracy_score(label_list, pred_list)
    end = time.time()
    
    print("## -- Training -- ##")
    print("Epoch: %d || Loss: %.3f +/- %.3f || Accuracy:  %.2f || Time: %.2f" % (epoch, epoch_loss.mean(),
                                                                                epoch_loss.std(), epoch_acc, end-start))
    
    return epoch_loss.mean()

In [40]:
def validate(test_loader, net, epoch, criterion):
    net.eval()
    start = time.time()
    
    epoch_loss = list()
    pred_list, label_list = np.array([]), np.array([])
    
    with torch.no_grad():
        for batch in test_loader:
            image, label = batch
            
            # Casting to GPU
            image = image.to(args["device"])
            label = label.to(args["device"])
            
            # Forward
            pred_label = net(image)
            loss = criterion(pred_label, label)
            epoch_loss.append(loss.cpu().data)
            
            _, pred = torch.max(pred_label, axis=1)
            pred_list = np.append(pred_list, pred.cpu().numpy())
            label_list = np.append(label_list, label.cpu().numpy())
        
    # Transforming to NumPy Array
    epoch_loss = np.asarray(epoch_loss)
    epoch_acc = accuracy_score(label_list, pred_list)
    end = time.time()

    print("*** Validate ***")
    print("Epoch: %d || Loss: %.3f +/- %.3f || Accuracy:  %.2f || Time: %.2f\n" % (epoch, epoch_loss.mean(), epoch_loss.std(), epoch_acc, end-start))
        
    return epoch_loss.mean()

In [42]:
train_losses_wea, test_losses_wea = list(), list()

for epoch in range(args["epoch_num"]):
    train_losses_wea.append(train(train_loader_wea, net_wea, epoch, criterion_wea, optimizer_wea))
    test_losses_wea.append(validate(test_loader_wea, net_wea, epoch, criterion_wea))

## -- Training -- ##
Epoch: 0 || Loss: 1.048 +/- 0.203 || Accuracy:  0.67 || Time: 28.71
*** Validate ***
Epoch: 0 || Loss: 0.777 +/- 0.144 || Accuracy:  0.78 || Time: 6.70

## -- Training -- ##
Epoch: 1 || Loss: 0.722 +/- 0.143 || Accuracy:  0.78 || Time: 6.38
*** Validate ***
Epoch: 1 || Loss: 0.695 +/- 0.204 || Accuracy:  0.76 || Time: 1.77

## -- Training -- ##
Epoch: 2 || Loss: 0.632 +/- 0.162 || Accuracy:  0.78 || Time: 6.22
*** Validate ***
Epoch: 2 || Loss: 0.661 +/- 0.235 || Accuracy:  0.78 || Time: 2.28

## -- Training -- ##
Epoch: 3 || Loss: 0.607 +/- 0.141 || Accuracy:  0.79 || Time: 5.78
*** Validate ***
Epoch: 3 || Loss: 0.905 +/- 0.203 || Accuracy:  0.64 || Time: 0.37

## -- Training -- ##
Epoch: 4 || Loss: 0.589 +/- 0.195 || Accuracy:  0.80 || Time: 5.98
*** Validate ***
Epoch: 4 || Loss: 0.563 +/- 0.181 || Accuracy:  0.80 || Time: 0.87

## -- Training -- ##
Epoch: 5 || Loss: 0.580 +/- 0.162 || Accuracy:  0.79 || Time: 6.48
*** Validate ***
Epoch: 5 || Loss: 0.639 +/- 0

KeyboardInterrupt: 

In [ ]:
# Real Test Values 
validation_images_wea = test_set_wea.images.to(args["device"]) 
validation_labels_wea = test_set_wea.labels

In [147]:
# Predict
y_pred_wea = np.array([np.argmax(net_wea(image.unsqueeze(0)).cpu().detach().numpy()) for image in validation_images_wea])

# Transform the numeric label back to string
validation_labels_wea = list(map(lambda x: unique_wea[x], validation_labels_wea))
y_pred_wea = list(map(lambda x: unique_wea[x], y_pred_wea))

In [152]:
y_pred_wea

540

---

In [ ]:
# crash-ego
train_dataset = CustomDataset(train['video_path'].values, train['crash_ego'].values) #, tfms=tfms
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=3)

val_dataset = CustomDataset(val['video_path'].values, val['crash_ego'].values) #, tfms=tfms
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=3)

# timing
train_dataset_time = CustomDataset(train['video_path'].values, train['timing'].values) #, tfms=tfms
train_loader_time = DataLoader(train_dataset_time, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=3)

val_dataset_time = CustomDataset(val['video_path'].values, val['timing'].values) #, tfms=tfms
val_loader_time = DataLoader(val_dataset_time, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=3)

# weather
train_dataset_wea = CustomDataset(train['video_path'].values, train['weather'].values) #, tfms=tfms
train_loader_wea = DataLoader(train_dataset_wea, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=3)

val_dataset_wea = CustomDataset(val['video_path'].values, val['weather'].values) #, tfms=tfms
val_loader_wea = DataLoader(val_dataset_wea, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=3)

#### Model Define

In [66]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=3):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(3, 8, (1, 3, 3)),
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.MaxPool3d(2),
            nn.Conv3d(8, 32, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(2),
            nn.Conv3d(64, 128, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d((3, 7, 7)),
        )
        self.classifier = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        
        return x

In [67]:
class BaseModel_wea(nn.Module):
    def __init__(self, num_classes=4):
        super(BaseModel_wea, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(3, 8, (1, 3, 3)),
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.MaxPool3d(2),
            nn.Conv3d(8, 32, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(2),
            nn.Conv3d(64, 128, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d((3, 7, 7)),
        )
        self.classifier = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

### Train

In [68]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [69]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

## Run!!

In [70]:
# crash-ego
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.74974] Val Loss : [0.56124] Val F1 : [0.60178]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

...

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.00670] Val Loss : [0.42368] Val F1 : [0.76777]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00638] Val Loss : [0.42379] Val F1 : [0.77435]


In [71]:
#time 
model_time = BaseModel()
model_time.eval()
optimizer_time = torch.optim.Adam(params = model_time.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_time = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_time, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model_time = train(model_time, optimizer_time, train_loader_time, val_loader_time, scheduler_time, device)

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.59737] Val Loss : [0.39989] Val F1 : [0.58386]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

...

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.00498] Val Loss : [0.24675] Val F1 : [0.79473]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.00474] Val Loss : [0.26919] Val F1 : [0.78035]
Epoch    20: reducing learning rate of group 0 to 9.3750e-06.


In [72]:
#weather
model_wea = BaseModel_wea()
model_wea.eval()
optimizer_wea = torch.optim.Adam(params = model_wea.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_wea = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_wea, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model_wea = train(model_wea, optimizer_wea, train_loader_wea, val_loader_wea, scheduler_wea, device)

  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.78279] Val Loss : [0.54034] Val F1 : [0.39823]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

...

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [19], Train Loss : [0.01075] Val Loss : [0.46219] Val F1 : [0.57195]


  0%|          | 0/270 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Epoch [20], Train Loss : [0.01026] Val Loss : [0.48954] Val F1 : [0.58073]
Epoch    20: reducing learning rate of group 0 to 1.8750e-05.


## Inference

In [73]:
test = pd.read_csv('./test.csv')

In [74]:
test_dataset = CustomDataset_Test(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=3)

In [75]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [76]:
preds = inference(model, test_loader, device)

  0%|          | 0/225 [00:00<?, ?it/s]

In [77]:
preds_time = inference(model_time, test_loader, device)

  0%|          | 0/225 [00:00<?, ?it/s]

In [78]:
preds_wea = inference(model_wea, test_loader, device)

  0%|          | 0/225 [00:00<?, ?it/s]

## Submission

In [79]:
submit = pd.read_csv('./sample_submission.csv')

In [80]:
submit['crash_ego']= preds

In [81]:
submit['timing']= preds_time

In [82]:
submit['weather']= preds_wea

In [83]:
submit['crash_ego'].value_counts()

0    1254
2     277
1     269
Name: crash_ego, dtype: int64

In [84]:
submit['timing'].value_counts()

0    1247
1     521
2      32
Name: timing, dtype: int64

In [85]:
submit['weather'].value_counts()

0    1259
1     492
2      45
3       4
Name: weather, dtype: int64

In [86]:
submit.to_csv('./sub/unrelabel/v2_unrelabel.csv',index=False)

---

In [87]:
submit.loc[submit[submit['crash_ego']==0].index,'weather'] = 0
submit.loc[submit[submit['crash_ego']==0].index,'timing'] = 0

In [88]:
submit.loc[(submit['weather']==1)*(submit['timing']==1),'label'] = 1
submit.loc[(submit['weather']==1)*(submit['timing']==2),'label'] = 2
submit.loc[(submit['weather']==2)*(submit['timing']==1),'label'] = 3
submit.loc[(submit['weather']==2)*(submit['timing']==2),'label'] = 4
submit.loc[(submit['weather']==3)*(submit['timing']==1),'label'] = 5
submit.loc[(submit['weather']==3)*(submit['timing']==2),'label'] = 6

submit.loc[submit.crash_ego==2,'label'] += 6

---

In [89]:
submit.label.value_counts()

0     1294
7      211
1      190
6       33
3       23
8       15
9       15
2       13
5        3
10       2
11       1
Name: label, dtype: int64

In [90]:
submit.iloc[:,:2].to_csv('./sub/v2_relabeled.csv',index=False)